# preparation
---

## enviroments

In [2]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf


## data

In [3]:
import urllib.request

url_base = 'http://yann.lecun.com/exdb/mnist/'
key_file = {
    'train_img':'train-images-idx3-ubyte.gz',
    'train_label':'train-labels-idx1-ubyte.gz',
    'test_img':'t10k-images-idx3-ubyte.gz',
    'test_label':'t10k-labels-idx1-ubyte.gz'
}

In [7]:
pwd = os.getcwd()
pwd

'c:\\Users\\redoa\\workspace\\projects\\first_vae'

In [9]:
dataset_dir = os.path.join(pwd, 'dataset')
dataset_dir

'c:\\Users\\redoa\\workspace\\projects\\first_vae\\dataset'

In [10]:
import os
for v in key_file.values():
    file_path = os.path.join(dataset_dir, v)
    if not os.path.exists(file_path):
        urllib.request.urlretrieve(url_base + v, file_path)
        
os.system('ls dataset')


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\redoa\\workspace\\projects\\first_vae\\dataset\\train-images-idx3-ubyte.gz'